In [4]:
from datasets import load_dataset

ds = load_dataset("hamedrahimi/Defect_Spectrum_cleaned")
ds

Using the latest cached version of the dataset since hamedrahimi/Defect_Spectrum_cleaned couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/hamed/.cache/huggingface/datasets/hamedrahimi___defect_spectrum_cleaned/default/0.0.0/0af4b9b32a3239c1c87fdfdb687e73359fa1dcfc (last modified on Thu May 29 00:34:06 2025).


DatasetDict({
    train: Dataset({
        features: ['image', 'mask_image', 'dataset', 'object', 'defect', 'object_description', 'defect_description'],
        num_rows: 3416
    })
})

In [6]:
from datasets import Dataset  # or load_dataset if needed
import math

columns_to_check = ["object_description", "defect_description"]  # replace with your actual column names

def is_valid(example):
    for col in columns_to_check:
        val = example[col]
        if val is None or (isinstance(val, float) and math.isnan(val)):
            return False
    return True

filtered_ds = ds.filter(is_valid)


Filter:   0%|          | 0/3416 [00:00<?, ? examples/s]

In [9]:
filtered_ds["train"][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024>,
 'mask_image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024>,
 'dataset': 'DS-MVTec',
 'object': 'grid',
 'defect': 'bent',
 'object_description': 'The image depicts a piece of metal grid.',
 'defect_description': 'In the upper left corner, the metal mesh is compressed and twisted, shaped like a net, in silver color. In the upper right corner, the metal mesh is similarly compressed and twisted, as well as in the lower left corner.'}

In [ ]:
# you need to log in to huggingface for having access to flux

from huggingface_hub import login

login(token=".....")

In [ ]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)

pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power

In [ ]:
#Testing

prompt= " ".join([filtered_ds["train"][0]["object_description"],filtered_ds["train"][0]["defect_description"]])

image = pipe(
    prompt,
    height=1024,
    width=1024,
    guidance_scale=3.5,
    num_inference_steps=50,
    max_sequence_length=512,
    generator=torch.Generator("cuda").manual_seed(0)
).images[0]

image.save("defect.png")